In [21]:
import brainlit
from brainlit.utils.session import NeuroglancerSession
from cloudvolume.lib import Bbox
import matplotlib.pyplot as plt
import numpy as np
import napari
from scipy import ndimage
from pathlib import Path
import skimage
import pytest
import glob
# %gui qt

brain_url = "s3://open-neurodata/brainlit/brain2"
segments_url = "s3://open-neurodata/brainlit/brain2_segments"
ngl_sess = NeuroglancerSession(mip=3, url=brain_url, url_segments=segments_url)
ImgPath = glob.glob('soma_detection/data/brain1/volumes/*.npy')

hit = np.zeros(1)    # True number = predicted number
miss = np.zeros(1)   # True number > prediceted number
FalsePos = np.zeros(1)   # True number < predicted number
TotalSoma = 0   # initiate total number of soma

NumSub = len(ImgPath)
for item in range(NumSub):
    print(item)
    volume_coords = np.array(os.path.basename(ImgPath[item]).replace('.npy','').split("_")).astype(float).round().astype(int).tolist()
    res = ngl_sess.cv_segments.scales[ngl_sess.mip]["resolution"]
    _min = volume_coords[:3]
    _max = volume_coords[3:]
    vox_min = np.round(np.divide(_min, res)).astype(int)
    vox_max = np.round(np.divide(_max, res)).astype(int)
    # obtain soma coordinate
    contained_coords = np.load(ImgPath[item])
    # true number of soma in subvolume
    TrueNumSo = len(contained_coords)
    bbox = Bbox(vox_min, vox_max)
    img = ngl_sess.pull_bounds_img(bounds=bbox)
    # with napari.gui_qt():
    #     viewer = napari.Viewer(ndisplay=3)
    #     napari.view_image(img)

    # run simple_soma_detector
    
    ### threshold top 5%
    H = np.sort(np.reshape(img,[1,-1]))
    thres = H[0,-round((H.shape[1])*0.05)]
    lx,ly,lz = img.shape
    imgTop5 = np.zeros([lx,ly,lz])
    for m in range(lx):
        for n in range(ly):
            for p in range(lz):
                if img[m,n,p] < thres:
                    imgTop5[m,n,p] = 0
                else:
                    imgTop5[m,n,p] = img[m,n,p]

    # viewer = napari.view_image(img, name='Original Image', colormap='red')
    # viewer.add_labels(imgTop5, name='Gray Threshold at 5%', num_colors=2, opacity=0.3)

    ### binary erosion
    ero5 = ndimage.binary_erosion(imgTop5, iterations=5)

    # viewer = napari.view_image(img, name='Original Image', colormap='red')
    # viewer.add_labels(ero5, name='Gray Erosion', num_colors=2, opacity=0.5)

    ### region label
    ReLab, NumLab = skimage.measure.label(ero5, return_num=True)
    # print('number of labels = ',NumLab)
    
    # visualize results
    # %gui qt
    # viewer = napari.view_image(img, name='Neuro'+str(item), colormap='green',ndisplay=3)
    # viewer.add_labels(ReLab, name='Region Labels', opacity=0.5)

    props = skimage.measure.regionprops(ReLab)
    CNumLab = NumLab
    for x in range(NumLab):
        D = props[x].equivalent_diameter
        # print('D=',D)
        # convert from voxel to micron
        Dmu = D*np.mean(np.array([res[0],res[1]]))/1000
        # print('Dmu=',Dmu)
        #print(D)
        if Dmu < 5 or Dmu > 21:
            CNumLab-=1

    print('Possible number of somas is',str(CNumLab))
    # count total number of soma
    TotalSoma += TrueNumSo
    # compare results
    Diff = CNumLab - TrueNumSo
    if Diff ==0:
        hit[0] = hit[0] + CNumLab
    elif Diff > 0:
        FalsePos[0] = FalsePos[0] + abs(Diff)
        hit[0] = hit[0] + TrueNumSo
    else:
        miss[0] = miss[0] + abs(Diff)
        hit[0] = hit[0] + CNumLab

print('Hit =',float(hit))
print('Miss =',float(miss))
print('False Positive=',float(FalsePos))
print('Hit Rate=',float(hit/TotalSoma)*100,'%')
print('Miss Rate=',float(miss/TotalSoma)*100,'%')
print('False Positive Rate=',float(FalsePos/TotalSoma)*100,'%')




    



Downloading:   0%|          | 0/2 [00:00<?, ?it/s]Possible number of somas is 1
1
Downloading:   0%|          | 0/1 [00:00<?, ?it/s]Possible number of somas is 0
2
Possible number of somas is 0
3

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]Possible number of somas is 0
4
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]Possible number of somas is 2
5
Possible number of somas is 2
6

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]
Possible number of somas is 1
7
Possible number of somas is 0
8
Downloading:   0%|          | 0/1 [00:00<?, ?it/s]
Possible number of somas is 2
9
Downloading:   0%|          | 0/2 [00:00<?, ?it/s]Possible number of somas is 0
10
Possible number of somas is 0
11

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]Possible number of somas is 1
12
Possible number of somas is 1
13
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]Possible number of somas is 1
14
Possible number of somas is 2
15
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]Possible 

## Collect locations of known somas

In [ ]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"
mip = 0; v_id = 0
voxs2 = []
ngl_sess = NeuroglancerSession(mip=mip, url=dir, url_segments=dir_segments)

for seg_id in range(1000):
    try:
        _,vox2,_ = ngl_sess.pull_voxel(seg_id, v_id)
        voxs2.append(np.array(np.floor(vox2.center()[:])))
    except:
        pass


In [ ]:
len(voxs2)

In [ ]:

Nbbox = [4689935,  2877908,  7506782.5,4789935.,  2977908.,  7606782.5]
KnoSomNum = 0
for vox in voxs2:
    if vox[0] > Nbbox[0] and vox[0] < Nbbox[3]:
        if vox[1] > Nbbox[1] and vox[1] < Nbbox[4]:
            if vox[2] > Nbbox[2] and vox[2] < Nbbox[5]:
                KnoSomNum +=1

print('Known number of soma within the subvolume is',str(KnoSomNum))


## Simple Soma Detector
### threshold top 5% followed by binary erosion

In [ ]:
%gui qt
### threshold top 5%
H = np.sort(np.reshape(img,[1,-1]))
thres = H[0,-round((H.shape[1])*0.05)]
lx,ly,lz = img.shape
imgTop5 = np.zeros([lx,ly,lz])
for m in range(lx):
    for n in range(ly):
        for p in range(lz):
            if img[m,n,p] < thres:
                imgTop5[m,n,p] = 0
            else:
                imgTop5[m,n,p] = img[m,n,p]

# viewer = napari.view_image(img, name='Original Image', colormap='red')
# viewer.add_labels(imgTop5, name='Gray Threshold at 5%', num_colors=2, opacity=0.3)

### binary erosion
ero5 = ndimage.binary_erosion(imgTop5, iterations=5)

# viewer = napari.view_image(img, name='Original Image', colormap='red')
# viewer.add_labels(ero5, name='Gray Erosion', num_colors=2, opacity=0.5)

### region label
ReLab, NumLab = skimage.measure.label(ero5, return_num=True)
print('number of labels = ',NumLab)

viewer = napari.view_image(img, name='Gray Erosion', colormap='green')
viewer.add_labels(ReLab, name='Region Labels', opacity=0.5)
# num_colors=20
props = skimage.measure.regionprops(ReLab)
CNumLab = NumLab
for x in range(NumLab):
    D = props[x].equivalent_diameter
    #print(D)
    if D < 10:
        CNumLab-=1

print('Possible number of somas is',str(CNumLab))


### Other methods

In [ ]:
## Binarize at top 0.1%
H = np.sort(np.reshape(img,[1,-1]))
thres = H[0,-round((H.shape[1])*0.001)]
lx,ly,lz = img.shape
imgBW = np.zeros([lx,ly,lz])
for m in range(lx):
    for n in range(ly):
        for p in range(lz):
            if img[m,n,p] < thres:
                imgBW[m,n,p] = 0
            else:
                imgBW[m,n,p] = 1

# viewer = napari.view_image(img, name='Original Image', colormap='red')
# viewer.add_labels(imgBW, name='Binary Threshold at 5%', num_colors=2, opacity=0.5)
## gray erosion
Gero = ndimage.grey_erosion(imgTop5, size=2)

viewer = napari.view_image(img, name='Original Image', colormap='red')
viewer.add_labels(Gero, name='Gray Erosion', num_colors=2, opacity=0.5)
